# Event detection in Twitter dataset
The goal of this project is to detect past events in Switzerland, using a dataset of tweets. This dataset contain 28 million tweets coming for most of them from Switzerland. This notebook will get you through our project, explaining our methodology. It is splitted in 5 parts:

1. **Data cleaning and hashtags extraction**<br>
We split the tweets
2. **Event detection**<br>

3. **Improving quality of detected event**<br>

4. **Exporting data**<br>


#### Importing useful libraries:

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
from datetime import timedelta
from operator import itemgetter
%matplotlib inline

Creation of check point (a print plus the time)

In [51]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)
    
def strNb(nb):
    return '{0:,}'.format(nb).replace(',', '.')

## 0 - Importing data

Importing a sample of the dataset:

In [47]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [4]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
# tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
#                       quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

19:11:10 Starting to read file... (3 min)
19:11:10 File is loaded!


In [52]:
print('The dataset contains {} tweets.'.format(strNb(len(tw))))

The dataset contains 100.000 tweets.


In [46]:
print('First rows of dataset:')
tw.head(2)

First rows of dataset:


,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,...,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland",[]
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland",[]


## 1 - Data cleaning and extracting hashtags

In [63]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

We are going to extract all the hashtags of the "text" cell in each tweet and put them in a new column (in the form of a list of hashtags per tweet).

In [65]:
pr('Extracting hashtags... (2 min)')
tw['hashtag'] = tw.text.apply(lambda x: extract_hashtags(str(x))) # Getting list of hashtag into new column
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)] # droping the rows (tweets) that contains no hashtags.
pr('We have found {} rows with hashtags.'.format(strNb(len(twh))))

19:23:58 Extracting hashtags... (2 min)
19:23:59 We have found 19.719 rows with hashtags.


In [61]:
print('Examples of tweets (with only text and hashtag column):')
twh[['text', 'hashtag']].head(2)

Examples of tweets (with only text and hashtag column):


,text,hashtag
8,"Magic spells run off after midnight, I guess s...",[fb]
10,"Limitas of public transportation! No taxi, rai...",[yam]


## Cleaning data and making date index

We drop tweet which not contains values for text or createdAt as this is mandatory information to privide to our model.

In [12]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 19719 tweets to 19719 tweets.


In [13]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

19:11:11 Removing bad dates...
19:11:11 Finished.


In [14]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

19:11:11 Starting to examine dates...
19:11:11 There are 0 dates that cannot be transformed.


In [15]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

19:11:11 Starting copy...
19:11:11 Converting to datetime...
19:11:11 Setting up new indices...
19:11:11 Deleting old "createdAt" column...
19:11:11 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [16]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [17]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [18]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

19:11:11 Multiplying the hashtag rows... (around 10 min)
19:11:13 Finished! 8887 rows will be added to the dataframe!


In [19]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

19:11:13 Starting to make the new dataframe with additionnal rows..
19:11:13 Starting to append the two df... Old df size = 19719
19:11:13 Done! New df size = 28606


In [20]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-02-23 17:47:11,1121,763.0,vfb,9535390586,NaN,47.6463,9.16570,NaN,NaN,NaN,eLd0raDo,550,Gowalla,http://gowalla.com/,6735.0,"So, Feierabend. Jetzt #24 und später #VfB. — a...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-12-01 17:25:56,1317,830.0,vfb,10021795398684672,NaN,46.9499,7.47071,e38a1a641d02f8db,46.9543,7.39491,chm,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,13552.0,"So, und jetzt Flachmann suchen und dann ab ins...",NaN,120433,"Bern, Switzerland",chm


In [21]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Grouping per hashtags per day

In order to provide visualisation to the other team, we needed to give ocation information to all tweets. So we did not process tweets without longitude or latitude.

In [22]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [23]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

Merging information of all tweet containing a particular hastag turned out to be really useful to detect an event. For column of String type, we used a join with a delimiter as balow.

In [24]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [25]:
def aggDate(df):
    temp = df.groupby(df.index.map(lambda x: x.date))
    groupedDf = temp.agg({  'text' : str_join,
                            'longitude' : np.median,
                            'latitude' : np.median,
                            'hashtag' : lambda x: x.iloc[0], ## the first occurence
                            'numberOfTweets' : 'count',
                            'userId' : pd.Series.nunique})
    # rename userId column to a more representative name
    return groupedDf

In [26]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

19:11:13 Making column number of tweets
19:11:13 Starting group by hastag...
19:11:13 Starting to put hashtag in dictionary... (around 50 min)
19:11:18 10%
19:11:23 20%
19:11:28 30%
19:11:33 40%
19:11:38 50%
19:11:43 60%
19:11:48 70%
19:11:53 80%
19:11:58 90%
19:12:03 100%
19:12:03 Finished operations! Dictionary with 9410 different hashtags.


In [27]:
# Creation of a dictionnary {'hashtag' : 'dataframe containing all tweet with the hashtag'}
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,userId,longitude,text,hashtag,latitude,numberOfTweets
2010-12-23,1,8.31025,schafft starbucks aus! Die reden immer englisc...,wirtschaftsfluechtling,47.056,1


## Event detection (with elimination of recurrent events)

Parameters that define events:

In [28]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 7 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2.5 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.
MAX_DURATION_OF_EVENT = timedelta(days=30) ## The maximum number of days we consider an event can happen
MIN_DURATION_BEFORE_NEW_EVENT = timedelta(days=304) ## (= 10 months) The min time that should pass before an event can happen
                                                    ## again and still be considered as event (ie. Christmas is an event
                                                    ## each year)
MIN_NUMBER_DIFFERENT_USER = 2 # To state that an event occured, a minimum number of different users should have tweeted about it

Helper functions to detect recurrent events that should be removed:

In [29]:
def isSpecificEventListIllegal(detectedEventDateList):
    '''
    Return true if the list of dates contain illegal tupples of events, so if the event is recurrent
    which would mean it is not a real event.
    '''
    def datesAreIllegal(date1, date2, date3):
        '''
        Return true if the 3 dates are not to be considered as regular events.
        '''
        ## Return if the difference is too small to be considered as 2 different events
        def diffIsSmall(timeDiff):  
            return timeDiff < MAX_DURATION_OF_EVENT

        ## Return true if the difference is not big enough to be an annual event.
        def isDiffSuspect(timeDiff):
            return timeDiff < MIN_DURATION_BEFORE_NEW_EVENT   

        diff1 = abs(date1 - date2)
        diff2 = abs(date2 - date3)
        diff3 = abs(date3 - date1)

        ## The difference is too small, it must be the same event
        if diffIsSmall(diff1) or diffIsSmall(diff2) or diffIsSmall(diff3):
            return False

        ## If there are at least 2 out of 3 suspect difference, then the dates are illegal
        if isDiffSuspect(diff1):
            return isDiffSuspect(diff2) or isDiffSuspect(diff3)
        else:
            return isDiffSuspect(diff2) and isDiffSuspect(diff3)
    
    ## MAIN FUNCTION : ##
    # Go through the list of events and try all "triples" to see if there is any illegal triples. This is a quickly done
    # code to do that. Code complexity bellow is in O(k^3), with k being the size of the list. We will apply this function
    # to n list so we will have an overall complexity in O(n*k^3). We can consider however that each list will
    # be small so k can be considered as constant and therefore the overall complexity will be in O(n).
    for i in range(len(detectedEventDateList) - 2):
        for j in range(i, len(detectedEventDateList) - 1):
            for k in range(j, len(detectedEventDateList)):
                if datesAreIllegal(detectedEventDateList[i], detectedEventDateList[j], detectedEventDateList[k]):
                    return True
    return False

Main method to detect event according to all our criterias.

In [30]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            if df['userId'][0] >= MIN_NUMBER_DIFFERENT_USER:
                threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
                df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

19:12:03 Starting to compute 9410 dict items to detect event. (4 min)
19:12:03 10%
19:12:04 20%
19:12:04 30%
19:12:04 40%
19:12:05 50%
19:12:05 60%
19:12:05 70%
19:12:06 80%
19:12:06 90%
19:12:06 100%
19:12:06 Finished! Number of events detected = 3


In [31]:
# pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
# nbOfEventDetected = 0
# count = 0
# printingValue = int(len(dictionary) / 10)
# for [h,df] in dictionary.items():
#     count += 1
#     if count % printingValue == 0:
#         pr("{:.0f}%".format(count/len(dictionary)*100))
#     df['event'] = False
#     if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
#         if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
#             threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
#             df['event'] = df.numberOfTweets.apply(lambda x: x >= threshold and x >= MIN_NB_TWEETS_DURING_EVENT)
            
#             ## Remove recurrent events:
#             detectedEventDf = df[df['event']]
#             if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
#                 df['event'] = False
#             nbOfEventDetected += len(df[df['event']])
# pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

## Merging close events and grouping into single event dataframe

Here, we have a function that is going to merge events that are considered as too "close" to each other to be considered individually.

In [32]:
def mergeCloseEvents(rowsList):
    '''
    Take a list of dictionary, where each dictionary is a "row" of the event df, which contained detected events.
    It will process the list to detect event that are close and merge them together.
    Return : the processed list of event.
    '''
    
    def areCloseEvents(event1, event2):
        '''
        Return true is 2 events dates are defined as "close"
        '''
        return abs(event1['date'] - event2['date']) < MAX_DURATION_OF_EVENT
        
    def mergeCloseEventsSublist(closeEventList):
        '''
        This will be applied to each close event sublist. It will merge all events into one unique event.
        The event will consist of the total number of tweets, with the concatenation of the tweet texts and the mean
        of longitude/latitude. A meanDate will be defined as a ponderated mean between all dates.
        The final date will be the one that is in the closeEventList and is closest to this mean date.
        We did this to keep the meaning of the date if it had some, and not have some meaningless "mean-date".
        '''
        latitude = 0
        longitude = 0
        numberOfTweets = 0
        text = ""
        originalDate = closeEventList[0]['date']
        dateDiff = timedelta(days=0)
        first = True        
        for tweet in closeEventList:
            longitude += tweet['longitude']
            latitude += tweet['latitude'] 
            numberOfTweets += tweet['numberOfTweets']
            if first:
                text = tweet['text']
                first = False
            else:
                text += delimiter + tweet['text']
                dateDiff = dateDiff + (tweet['date'] - originalDate) * tweet['numberOfTweets']

        ## It is multiplied by 2 then soustracted to round correctly to the nearest day
        meanDate = originalDate + 2* dateDiff / numberOfTweets - dateDiff / numberOfTweets        
        latitude = latitude / len(closeEventList)
        longitude = longitude / len(closeEventList)
        
        ## We are going to detect the event the closest to the mean date
        minSelectedDate = closeEventList[0]['date']
        minDistance = abs(closeEventList[0]['date'] - meanDate)
        for tweet in closeEventList:
            if abs(tweet['date'] - meanDate) < minDistance:
                minSelectedDate = tweet['date']    
        
        return {'date': minSelectedDate, 'hashtag': closeEventList[0]['hashtag'], 'text': text,
                    'longitude': longitude, 'latitude':latitude, 'numberOfTweets': numberOfTweets, }
    
    ############ -----  MAIN METHOD  ----- ############
    
    ## If the list is big enough, go through the list and form an export list and merge elements that needs to.
    if len(rowsList) < 2:
        return rowsList
    else:
        firstLastPosOfItemsToMerge = []
        sortedRowsList = sorted(rowsList, key=itemgetter('date')) 
        exportedEventList = []
        ## This goes through the *sorted* list and add the pair of indices (first indice and last indice) where events 
        ## that should be merged appear.
        lastEventWasClose = False
        firstItem = -1
        for i in range(0, len(sortedRowsList)-1):
            if areCloseEvents(sortedRowsList[i], sortedRowsList[i+1]):
                if not lastEventWasClose: # So it is the first pairs of the sublist of close events in the whole list
                    firstItem = i
                    lastEventWasClose = True
            else:
                if lastEventWasClose: # So the list has just ended.
                    exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:i+1]))
                    lastEventWasClose = False
                else: # The element is by itself, let's append it
                    exportedEventList.append(sortedRowsList[i])  
        if lastEventWasClose: # If there were events to merge till the last elem of list
            exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:len(sortedRowsList)]))
        else:
            exportedEventList.append(sortedRowsList[len(sortedRowsList)-1])
    
    return exportedEventList

This function will be applied to each dataframe. If a row is detected as an event, it will be added to the locaRowsList which will be used to make a general dataframe of all the events.

In [33]:
localRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        global localRowsList
        localRowsList.append(rowToAdd)

In [34]:
eventRowsList = []
localRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    global localRowsList
    localRowsList = []
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
        
    df.apply(applyToMakeEventDf, axis=1)
    mergedList = mergeCloseEvents(localRowsList) # merging close events
    eventRowsList += mergedList

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

19:12:07 Starting to make event df with 9410 dataframes. (around 6 min)
19:12:07 10%
19:12:08 20%
19:12:08 30%
19:12:09 40%
19:12:09 50%
19:12:10 60%
19:12:10 70%
19:12:11 80%
19:12:11 90%
19:12:12 100%
19:12:12 Making new dataframe.
19:12:12 Finished! Dataframe with 3 rows


In [35]:
print('Events dataframe:')
new_events.head(50)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2011-03-25,ipad2,47.3751,8.53896,41,@greezer der @ThBenkoe hat's schon #iPad2 ht...
2010-06-16,sui,47.3788,8.53803,25,"#sui wonderful_$$$_en plus du goal, j'aurai mi..."
2011-06-08,ios5,47.4850,8.29721,9,@AndreBonhote einstellungen gemacht? #ios5 ht...


In [36]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,userId,longitude,text,hashtag,latitude,numberOfTweets,event
2011-03-02,3,8.42875,http://bit.ly/hpQGAV #ipad2 #leaked photos_$$$...,ipad2,47.0333,3,False
2011-03-04,1,9.60857,Conan takes on the #iPad2 http://t.co/I97hmyJ,ipad2,47.2488,1,False
2011-03-15,1,8.51569,@pokipsie oh! #ipad2 schon bestellt?,ipad2,47.3601,1,False
2011-03-25,12,8.53896,@greezer der @ThBenkoe hat's schon #iPad2 ht...,ipad2,47.3751,41,True
2011-03-26,1,8.50014,Maaaaannnn after a Day like this I'm to pumped...,ipad2,47.3689,1,False
2011-04-06,1,8.54340,"ach wie schön, nur ein paar Kilometer von Züri...",ipad2,47.2872,1,False
2011-04-08,1,8.51900,@rkroebl Fehlplanung. Habs verkauft um dann di...,ipad2,47.1827,1,False
2011-06-08,1,8.29843,Ich suche ein #ipad2 64Gb 3G. Wer verkauft ein...,ipad2,47.4842,1,False
2011-06-15,1,8.29995,Wär hätte Interesse an neuem #iPad2 64GB 3G ? ...,ipad2,47.4693,1,False
2011-09-24,1,8.26805,"@greezer mh, 4.3.3 lässt sich noch jailbreaken...",ipad2,47.2089,1,False


## Exporting data

As we worked with another team, we needed a way to communicate them our detection. We used a JSON with all the information.

In [37]:
total_number_of_events = len(new_events)
print('There are {} events.'.format(total_number_of_events))

There are 3 events.


In [38]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,ipad2,47.3751,8.53896,41,@greezer der @ThBenkoe hat's schon #iPad2 ht...,2011-03-25


We are going to generate the right datetimes for the jsons:

In [39]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [40]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [41]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

19:12:12 Converting dates...
19:12:12 Done.


,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,ipad2,47.3751,8.53896,41,@greezer der @ThBenkoe hat's schon #iPad2 ht...,2011-03-25,2011,3,1301011200000,1298937600000


The generation of a JSON is easier from a dictionary than from a dataframe. Also, the other team we worked with asked us to group events by months.

In [42]:
# Grouping by months
e_gb_month = e_df.groupby(e_df.unix_time)

In [43]:
# Generation of the dictionary for the final JSON
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

19:12:12 Making event list...
19:12:12 Done.


Creation of the final JSON

In [44]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + \
'_' + str(total_number_of_events)+ '_events.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))

19:12:12 Exporting to json...
19:12:12 Export done. File "export_twitter_events_2017-02-03_19h12min12_3_events.json" has been created.
